In [2]:
import numpy as np
import cv2
import pandas as pd

In [80]:
help(cv2.threshold)

Help on built-in function threshold:

threshold(...)
    threshold(src, thresh, maxval, type[, dst]) -> retval, dst
    .   @brief Applies a fixed-level threshold to each array element.
    .   
    .   The function applies fixed-level thresholding to a multiple-channel array. The function is typically
    .   used to get a bi-level (binary) image out of a grayscale image ( #compare could be also used for
    .   this purpose) or for removing a noise, that is, filtering out pixels with too small or too large
    .   values. There are several types of thresholding supported by the function. They are determined by
    .   type parameter.
    .   
    .   Also, the special values #THRESH_OTSU or #THRESH_TRIANGLE may be combined with one of the
    .   above values. In these cases, the function determines the optimal threshold value using the Otsu's
    .   or Triangle algorithm and uses it instead of the specified thresh.
    .   
    .   @note Currently, the Otsu's and Triangle methods a

In [96]:
#Read the image
base = '/mnt/c/Users/Михаил/Documents/IITP/ImgProc/Pics/'
im = cv2.imread(base + 'skyscraper2.jpg')
gray = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
equ = cv2.equalizeHist(gray)
#ret, thresh = cv2.threshold(equ,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
winSize = 7
blured = cv2.GaussianBlur(equ, (winSize, winSize), 0)
edges = cv2.Canny(blured,100,220,apertureSize = 7, L2gradient=True)



#cv2.imshow('houghlines',im)
cv2.imwrite(base+'gray.jpg', gray)
cv2.imwrite(base+'equ.jpg', equ)
cv2.imwrite(base+'blured.jpg', blured)
cv2.imwrite(base+'edges.jpg', edges)

#cv2.imwrite(base+'thresh.jpg', thresh)

#A few changes to check git functionality

True

In [44]:
help(cv2.Canny)

Help on built-in function Canny:

Canny(...)
    Canny(image, threshold1, threshold2[, edges[, apertureSize[, L2gradient]]]) -> edges
    .   @brief Finds edges in an image using the Canny algorithm @cite Canny86 .
    .   
    .   The function finds edges in the input image and marks them in the output map edges using the
    .   Canny algorithm. The smallest value between threshold1 and threshold2 is used for edge linking. The
    .   largest value is used to find initial segments of strong edges. See
    .   <http://en.wikipedia.org/wiki/Canny_edge_detector>
    .   
    .   @param image 8-bit input image.
    .   @param edges output edge map; single channels 8-bit image, which has the same size as image .
    .   @param threshold1 first threshold for the hysteresis procedure.
    .   @param threshold2 second threshold for the hysteresis procedure.
    .   @param apertureSize aperture size for the Sobel operator.
    .   @param L2gradient a flag, indicating whether a more accurate \

In [13]:
help(cv2.HoughLines)

Help on built-in function HoughLines:

HoughLines(...)
    HoughLines(image, rho, theta, threshold[, lines[, srn[, stn[, min_theta[, max_theta]]]]]) -> lines
    .   @brief Finds lines in a binary image using the standard Hough transform.
    .   
    .   The function implements the standard or standard multi-scale Hough transform algorithm for line
    .   detection. See <http://homepages.inf.ed.ac.uk/rbf/HIPR2/hough.htm> for a good explanation of Hough
    .   transform.
    .   
    .   @param image 8-bit, single-channel binary source image. The image may be modified by the function.
    .   @param lines Output vector of lines. Each line is represented by a two-element vector
    .   \f$(\rho, \theta)\f$ . \f$\rho\f$ is the distance from the coordinate origin \f$(0,0)\f$ (top-left corner of
    .   the image). \f$\theta\f$ is the line rotation angle in radians (
    .   \f$0 \sim \textrm{vertical line}, \pi/2 \sim \textrm{horizontal line}\f$ ).
    .   @param rho Distance resolution

In [36]:
im.shape

(270, 360, 3)

In [133]:
def draw_line (img, rho, theta, color=(0,255,0)):
    a = np.cos(theta)
    b = np.sin(theta)
    x0 = a*rho
    y0 = b*rho
    x1 = int(x0 + 1000*(-b))   # Here i have used int() instead of rounding the decimal value, so 3.8 --> 3
    y1 = int(y0 + 1000*(a))    # But if you want to round the number, then use np.around() function, then 3.8 --> 4.0
    x2 = int(x0 - 1000*(-b))   # But we need integers, so use int() function after that, ie int(np.around(x))
    y2 = int(y0 - 1000*(a))
    cv2.line(img,(x1,y1),(x2,y2),color,2)
    print (rho, theta, color)

In [124]:
np.pi / 2

1.5707963267948966

In [136]:
img = im.copy()
rho_minus = 0
rho_plus = np.inf
rho_hor_max = 0
rho_hor_min = np.inf
eps = 0.01
lines = cv2.HoughLines(edges,1,np.pi/180, int (min(im.shape[0], im.shape[1]) / 2))
#print (lines)
for line in lines:
    for rho,theta in line:
        if (theta > np.pi / 2 + eps and np.abs (rho) > np.abs (rho_minus)):
            rho_minus = rho
            theta_minus = theta
        if (theta < np.pi / 2 - eps and rho < rho_plus):
            rho_plus = rho
            theta_plus = theta
        if (np.abs (theta - np.pi / 2) < eps):
            if (rho > rho_hor_max):
                rho_hor_max = rho
            if (rho < rho_hor_min):
                rho_hor_min = rho
        draw_line (img, rho, theta)
red = (0, 0, 255)
draw_line (img, rho_minus, theta_minus, red)
draw_line (img, rho_plus, theta_plus, red)
draw_line (img, rho_hor_min, np.pi / 2, red)
draw_line (img, rho_hor_max, np.pi / 2, red)
cv2.imwrite(base+'houghlines.jpg', img)
#cv2.imshow('houghlines',img)

-8.0 2.1642082 (0, 255, 0)
234.0 1.5707964 (0, 255, 0)
-169.0 2.687807 (0, 255, 0)
193.0 0.9599311 (0, 255, 0)
210.0 1.5707964 (0, 255, 0)
228.0 1.5707964 (0, 255, 0)
129.0 0.715585 (0, 255, 0)
197.0 1.5707964 (0, 255, 0)
-143.0 2.443461 (0, 255, 0)
201.0 1.5707964 (0, 255, 0)
223.0 1.5707964 (0, 255, 0)
154.0 0.4537856 (0, 255, 0)
174.0 1.553343 (0, 255, 0)
245.0 1.5707964 (0, 255, 0)
185.0 1.553343 (0, 255, 0)
265.0 1.5707964 (0, 255, 0)
-142.0 2.4609141 (0, 255, 0)
197.0 0.94247776 (0, 255, 0)
171.0 1.5707964 (0, 255, 0)
159.0 1.553343 (0, 255, 0)
181.0 1.553343 (0, 255, 0)
178.0 1.5707964 (0, 255, 0)
-165.0 2.687807 (0, 255, 0)
189.0 1.5707964 (0, 255, 0)
245.0 0.7853982 (0, 255, 0)
-49.0 2.3561945 (0, 255, 0)
-6.0 2.3561945 (0, 255, 0)
152.0 1.553343 (0, 255, 0)
-132.0 2.4085543 (0, 255, 0)
-169.0 2.687807 (0, 0, 255)
129.0 0.715585 (0, 0, 255)
171.0 1.5707963267948966 (0, 0, 255)
265.0 1.5707963267948966 (0, 0, 255)


True

In [113]:
img = im.copy()
lines = cv2.HoughLinesP(edges,1,np.pi/180,80, minLineLength = 100, maxLineGap = 10)
for line in lines:
    for x1,y1,x2,y2 in line:
        cv2.line(img,(x1,y1),(x2,y2),(0,255,0),2)
cv2.imwrite(base+'houghlines_p.jpg', img)
#cv2.imshow('houghlines',img)


True